
##### Extra : Random Forest 

Dataset disponibles :
- X_train : avec valeurs Nan , sans transformation (Hyperplot+300 -> 0.677)
- X_train_forest_imputed : Nan imputés avec Forest (Hyperplot+300 -> 0.673)
- X__Mice_imputed : Nan imputés avec Mice  (Hyperplot+300 -> 0.672)
- X_poly_2_No_nan : Nan imputés avec Forest + polynomes et interractions  (Hyperplot+300 -> 0.657)
- X_train_RandomUnderSampler , y_train_RandomUnderSampler : données parfaitement équilibrées mais moins importantes. (Hyperplot+300 -> 0.65) 
- X_train_del : imputation will bring a lot of noise (Hyperplot+300 -> 0.672, 1000 -> 0.673)
- X_train_poly2_outnan : polynomes sans OP100 (nan values) (Hyperplot+300 -> 0.657)

Les tests nous montrent que quelque soit la méthode d'imputation retenue , il y a perte de performances. Ceci est très probablement du  au fait que la création de bruit par les méthodes d'imputation.

In [20]:
import pandas as pd
import numpy as np
data_x = pd.read_csv('traininginputs.csv')
data_y = pd.read_csv('trainingoutput.csv')
data = pd.merge(data_x, data_y, on='PROC_TRACEINFO', how ='inner')
data = data.drop('PROC_TRACEINFO', axis =1 )
data['Binar OP130_Resultat_Global_v'] = data['Binar OP130_Resultat_Global_v'].astype('category')
data = data.dropna(subset=['OP100_Capuchon_insertion_mesure'])
X_train = data.drop('Binar OP130_Resultat_Global_v', axis =1)
y_train = data['Binar OP130_Resultat_Global_v']


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [22]:
import optuna
import warnings
warnings.filterwarnings('ignore')
from optuna import Trial, create_study, study

In [24]:
# Créer un objet Optuna Study
study = create_study(direction="maximize")

class_weights = {0: 1, 1: 1}

# Définition de l'espace de recherche
def objective(trial: Trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 2000)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    min_weight_fraction_leaf = trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5)


    # Créer un modèle RandomForestClassifier avec les paramètres suggérés
    clf = RandomForestClassifier(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  min_weight_fraction_leaf= min_weight_fraction_leaf,
                                  class_weight = class_weights, 
                                  n_jobs = -1)
    
    # Évaluation à l'aide de la validation croisée avec ROC AUC comme métrique
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    return scores.mean()

# Création de l'étude Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Affichage des meilleurs hyperparamètres et de la meilleure ROC AUC moyenne
best_params = study.best_params
best_auc = study.best_value

[I 2023-11-04 15:30:21,339] A new study created in memory with name: no-name-f261dd97-a468-4bf4-92ec-197df3249a09
[I 2023-11-04 15:30:21,341] A new study created in memory with name: no-name-27771be3-adc2-4f34-8b46-9a32f9c6ea36
[I 2023-11-04 15:30:22,620] Trial 0 finished with value: 0.6444200958119483 and parameters: {'n_estimators': 126, 'max_depth': 1, 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.20465821461164196}. Best is trial 0 with value: 0.6444200958119483.
[I 2023-11-04 15:30:22,988] Trial 1 finished with value: 0.6391626438202633 and parameters: {'n_estimators': 60, 'max_depth': 2, 'min_samples_split': 10, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0.24421403869311042}. Best is trial 0 with value: 0.6444200958119483.
[I 2023-11-04 15:30:26,241] Trial 2 finished with value: 0.63865868567135 and parameters: {'n_estimators': 1068, 'max_depth': 2, 'min_samples_split': 8, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.334020616169270

In [25]:
best_auc

0.6556958216088982

In [ ]:
optuna.visualization.plot_optimization_history(study)